# LangChain: Models, Prompts and Output Parsers


## Outline

 * API calls through LangChain:
   * Simple Calls
   * Prompts
   * Models
   * Output parsers

## Setting Up

In [0]:
!pip install -qU openai
!pip install -qU langchain 
!pip install -qU langchain-openai
!pip install -qU tiktoken
dbutils.library.restartPython()

## Exploring Azure OpenAI

#### Let's start by comparing model responses with direct API calls to Azure OpenAI.

In [0]:
import os
import openai
from openai import OpenAI
from openai import AzureOpenAI

openai.api_type = "azure"
azure_endpoint = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#azure_endpoint = "https://chatgpt-summarization.openai.azure.com/"

openai.api_version = "2023-07-01-preview"
openai.api_key = os.environ["OPENAI_API_KEY"]
deployment_name = "model-gpt-35-turbo"
openai_model_name = "gpt-35-turbo"

client = AzureOpenAI(api_key=openai.api_key,
                     api_version=openai.api_version,
                     azure_endpoint=azure_endpoint,
                     )

def ask(prompt):
    try:
        chat_completion = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ],
            )
        return(chat_completion.choices[0].message.content)
    except openai.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
    except openai.error.AuthenticationError as e:
        print(f"OpenAI API returned an Authentication Error: {e}")
    except openai.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
    except openai.error.InvalidRequestError as e:
        print(f"Invalid Request Error: {e}")
    except openai.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
    except openai.error.ServiceUnavailableError as e:
        print(f"Service Unavailable: {e}")
    except openai.error.Timeout as e:
        print(f"Request timed out: {e}")
    except:
        print("An exception has occured.")

#### Simple queries to the models:  

In [0]:
response = ask("Can you tell me how much is 1+1?")
print(response)

#### Queries with custom prompts using formatted strings  

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [0]:
response = ask(prompt)
print(response)

### Using Langchain Wrappers

LangChain is an open-source framework that gives developers the tools they need to create applications using large language models (LLMs). In its essence, LangChain is a prompt orchestration tool that makes it easier for teams to connect various prompts interactively.  

Let's import the necessary libraries and examine some use cases.  
Remember the necessary imports if we are either using ChatGPT or AzureChatGPT:  

In [0]:
import os
import tiktoken
import openai

#from openai import OpenAI
#from openai import AzureOpenAI
del OpenAI
del AzureOpenAI

from langchain.llms import OpenAI
from langchain_openai import AzureOpenAI

from langchain.chat_models import ChatOpenAI
from langchain_openai import AzureChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

openai.api_type = "azure"
azure_endpoint = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#azure_endpoint = "https://chatgpt-summarization.openai.azure.com/"

openai.api_version = "2023-07-01-preview"
openai.api_key = os.environ["OPENAI_API_KEY"]

deployment_name = "model-gpt-35-turbo"
openai_model_name = "gpt-35-turbo"

client = AzureOpenAI(api_key=openai.api_key,
                     api_version=openai.api_version,
                     azure_endpoint=azure_endpoint,
                     )

chat = AzureChatOpenAI(azure_endpoint=azure_endpoint,
                       openai_api_version=openai.api_version,
                       deployment_name=deployment_name,
                       openai_api_key=os.environ["OPENAI_API_KEY"],
                       openai_api_type=openai.api_type,
                       )

### Using the Prompt Template

In [0]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [0]:
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

##### The prompt Template allow us to have more control of the input variables, instead of relying on Python Formatted Strings  

In [0]:
prompt_template.messages[0].prompt.input_variables

##### Now we can shape our prompts just by injecting the variables that were defined

In [0]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [0]:
customer_messages = prompt_template.format_messages(style=customer_style,
                                                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

##### And we can call the LLM with the formatted prompt template to translate to the style of the customer message  

In [0]:
customer_response = chat(customer_messages)
print(customer_response.content)

##### Another example  

In [0]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [0]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [0]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [0]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers

Output parsers help us enforcing an output format for our data  
Let's start with defining how we would like the LLM output to look like:  

```
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [0]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [0]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)

print(type(response.content))
print(response.content)

##### Apparently we have a JSON response, but it is actually a string  

You will get an error by running this line of code because'gift' is not a dictionary; 'gift' is a string

In [0]:
try:
  response.content.get('gift')
except AttributeError as error:
    print(error)

### Parse the LLM output string into a Python dictionary using Response Schema  

In [0]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this information is not found, output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [0]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

In [0]:
response = chat(messages)
print(response.content)

In [0]:
output_dict = output_parser.parse(response.content)
print(type(output_dict))
output_dict

In [0]:
output_dict.get('delivery_days')